In [2]:
import torch
from torch.utils.data import DataLoader

In [3]:
from pathlib import Path
import yaml

In [5]:
import sys
sys.path.append("..")
sys.path.append("../source")
from helper import init_model
from datasets.ptz_dataset import PTZImageDataset

INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [25]:
with open("../configs/Config_file.yaml", 'r') as y_file:
    args = yaml.load(y_file, Loader=yaml.FullLoader)
use_bfloat16 = args['meta']['use_bfloat16']
model_name = args['meta']['model_name']
pred_depth = args['meta']['pred_depth']
pred_emb_dim = args['meta']['pred_emb_dim']
patch_size = args['mask']['patch_size']  # patch-size for model training
crop_size = args['data']['crop_size']
batch_size = args['data']['batch_size']
camera_brand = args['meta']['camera_brand']


In [7]:
encoder, predictor = init_model(
        device="cpu",
        patch_size=patch_size,
        crop_size=crop_size,
        pred_depth=pred_depth,
        pred_emb_dim=pred_emb_dim,
        model_name=model_name)

INFO:root:VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 192, kernel_size=(14, 14), stride=(14, 14))
  )
  (blocks): ModuleList(
    (0-11): 12 x Block(
      (norm1): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=192, out_features=576, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=192, out_features=192, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
      (mlp): MLP(
        (fc1): Linear(in_features=192, out_features=768, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=768, out_features=192, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
  )
  (norm): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
)


In [76]:
import importlib
import source
importlib.reload(source)
from source.run_jepa import get_position_from_label, forward_context, arrage_inputs

In [77]:
from source.datasets.ptz_dataset import PTZImageDataset

In [78]:
from torchvision.transforms import ToTensor

In [92]:
from source.transforms import make_transforms

In [93]:
data = PTZImageDataset('../../collected_imgs_bu2', transform=make_transforms())
dataloader = DataLoader(data, batch_size=batch_size, shuffle=False)
ipe = len(dataloader)

INFO:root:making ptz image data transforms


In [94]:
img_lab = next(iter(dataloader))

In [95]:
isinstance(img_lab[1], Iterable)

True

In [96]:
pos = get_position_from_label(img_lab[1])

In [97]:
context_imgs, context_poss, target_imgs, target_poss = arrage_inputs(img_lab[0], pos, "cpu")

In [100]:
z, r = forward_context(context_imgs, context_poss, target_poss, encoder, predictor, camera_brand)

In [101]:
z.shape

torch.Size([16, 256, 192])